## Ejercicio 5

### Generación de muestra normal

In [ ]:
ks_stat_norm, ks_pval_norm = stats.ks_2samp(teor_norm, emp_norm)
print("Normal — KS two-sample: stat={:.4f}, p={:.4f}".format(ks_stat_norm, ks_pval_norm))

# Chi2 binned: histogram de la muestra empírica y probabilidades teóricas por bin
n_bins = 20
obs_counts, bin_edges = np.histogram(emp_norm, bins=n_bins)
cdf_vals = stats.norm.cdf(bin_edges, loc=mu, scale=sigma)
expected_probs = np.diff(cdf_vals)
expected_counts = expected_probs * N

# Agrupar/ajustar si expected_counts pequeños y asegurar suma
mask_small = expected_counts < 5
if mask_small.any():
    idx_non_small = np.where(~mask_small)[0]
    if len(idx_non_small) == 0:
        raise ValueError("Binning llevó expected <5 en todas las celdas. Aumenta N o reduce n_bins.")
    keep = idx_non_small[-1]
    observed_b = np.concatenate([obs_counts[:keep], [obs_counts[keep:].sum()]])
    expected_b = np.concatenate([expected_counts[:keep], [expected_counts[keep:].sum()]])
else:
    observed_b = obs_counts.astype(float)
    expected_b = expected_counts.astype(float)

# Ajuste para que sumas coincidan (corrige redondeos)
diff = observed_b.sum() - expected_b.sum()
if abs(diff) > 1e-8 * max(1.0, observed_b.sum()):
    expected_b[-1] += diff

# Chequeo final
if not np.isclose(observed_b.sum(), expected_b.sum(), rtol=1e-8, atol=1e-12):
    raise ValueError("Suma de esperados no coincide con observados tras ajuste (Normal).")

chi2_stat_norm, chi2_pval_norm = stats.chisquare(observed_b, f_exp=expected_b)
print("Normal — Chi2 (binned): stat={:.3f}, p={:.4f}".format(chi2_stat_norm, chi2_pval_norm))
print("Suma observados (binned) = {}, suma esperados ajustados = {}".format(observed_b.sum(), expected_b.sum()))

# Visualizaciones
fig, axes = plt.subplots(1,2, figsize=(14,4))
axes[0].hist(teor_norm, bins=50, density=True, alpha=0.6, label='teórica (scipy)')
axes[0].hist(emp_norm,  bins=50, density=True, alpha=0.4, label='empírica (transformada)')
xs = np.linspace(mu-4*sigma, mu+4*sigma, 300)
axes[0].plot(xs, stats.norm.pdf(xs, mu, sigma), 'k--', lw=2, label='pdf teórica')
axes[0].legend()
axes[0].set_title('Normal — densidades')

# QQ plot entre las dos muestras (visual)
s_te = np.sort(teor_norm)
s_em = np.sort(emp_norm)
m = min(len(s_te), len(s_em))
axes[1].scatter(s_te[:m], s_em[:m], s=6)
axes[1].plot([s_te[:m].min(), s_te[:m].max()], [s_te[:m].min(), s_te[:m].max()], 'r--')
axes[1].set_title('QQ-plot (teórica vs empírica)')
plt.show()


In [ ]:
alpha = 0.05

print("=== Interpretación propuesta (α = {:.2f}) ===".format(alpha))
print("Geométrica: Chi2 (empírica) p = {:.4f} -> {}".format(
    pval_chi2_e, "NO rechazar H0" if pval_chi2_e > alpha else "Rechazar H0"))
print("Geométrica: KS two-sample p = {:.4f} -> {}".format(
    ks_pval, "NO rechazar H0" if ks_pval > alpha else "Rechazar H0"))

print("\nNormal: KS two-sample p = {:.4f} -> {}".format(
    ks_pval_norm, "NO rechazar H0" if ks_pval_norm > alpha else "Rechazar H0"))
print("Normal: Chi2 (binned) p = {:.4f} -> {}".format(
    chi2_pval_norm, "NO rechazar H0" if chi2_pval_norm > alpha else "Rechazar H0"))

